In [1]:
%reset -sf

In [2]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
# d8 = [(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [3]:
# trr = ""
# srr = """
# """.strip()


# trr = ""
with open("/Users/hkmac/Downloads/input.txt") as f:
    srr_inp = f.read().strip()

# with open("input1.txt") as f:
#     srr_inp = f.read().strip()
    
srr_inp = list(srr_inp.split("\n\n"))
srr_inp = [srr_in.strip().split("\n")[1:] for srr_in in srr_inp]
srr_inp = [[list(map(int,srr_i.split(","))) for srr_i in srr_in] for srr_in in srr_inp]

In [4]:
len(srr_inp)

32

In [5]:
import copy

mapping = {}

for ii in range(len(srr_inp)):
    for jj in range(len(srr_inp)):
        if ii == jj:
            continue
        
        arr = srr_inp[ii]
        brr = srr_inp[jj]

        for px,py,pz in itertools.product([-1,1], repeat=3):
            for transform in [
    lambda x: (x[0],x[1],x[2]),
    lambda x: (x[0],x[2],x[1]),
    lambda x: (x[1],x[0],x[2]),
    lambda x: (x[1],x[2],x[0]),
    lambda x: (x[2],x[0],x[1]),
    lambda x: (x[2],x[1],x[0]),
]:
                xrr = [transform((x*px,y*py,z*pz)) for x,y,z in arr]

                for (p,q,r) in (xrr):
                    for (p2,q2,r2) in (brr):
                        p9,q9,r9 = p-p2, q-q2, r-r2
                        
                        tf = lambda x: (transform((x[0]*px,x[1]*py,x[2]*pz),)[0] - p9, 
                                        transform((x[0]*px,x[1]*py,x[2]*pz),)[1] - q9, 
                                        transform((x[0]*px,x[1]*py,x[2]*pz),)[2] - r9)
                        
                        prr = [tf((x,y,z),) for x,y,z in arr]
                        qrr = [(x,y,z) for x,y,z in brr]

                        if len(set(prr) & set(qrr)) >= 12:
                            mapping[ii,jj] = (transform, px, py, pz, p9, q9, r9)
                            print(ii,jj,"ok")
                            print(id(tf))
                            print(tf((1,2,3),))
                            break
                    if (ii,jj) in mapping:
                        break
                if (ii,jj) in mapping:
                    break
            if (ii,jj) in mapping:
                break

0 9 ok
140442898593104
(-1313, -29, -36)
0 17 ok
140442898594688
(1210, -81, -129)
0 21 ok
140442898594544
(-1192, 88, 8)
1 4 ok
140442898594400
(-108, 73, -1109)
1 7 ok
140442898595408
(1126, -86, -158)
1 16 ok
140442898594688
(-1233, -111, 59)
1 21 ok
140442898595696
(1203, 86, -103)
2 12 ok
140442898594112
(-14, -50, 1122)
2 17 ok
140442898594112
(-1206, -10, -115)
2 20 ok
140442898595696
(-67, -1199, -55)
3 6 ok
140442898595696
(-55, 1316, 57)
3 13 ok
140442898593680
(1175, -45, -84)
3 23 ok
140442898594688
(-7, -16, 1250)
3 24 ok
140442898593968
(185, 1189, 54)
3 28 ok
140442898593824
(-122, -1250, -70)
4 1 ok
140442898593392
(110, 1114, 74)
4 6 ok
140442898594688
(1193, -49, 110)
4 23 ok
140442898593968
(1358, 37, 2)
5 7 ok
140442898593968
(-3, -1166, -192)
5 15 ok
140442898593392
(1382, 86, 124)
5 22 ok
140442898593392
(178, -156, -1145)
5 26 ok
140442898593392
(132, 1230, 58)
6 3 ok
140442898593392
(-53, -1312, -53)
6 4 ok
140442898593392
(-50, 1194, 110)
6 8 ok
140442898593392

In [6]:
for k,func in mapping.items():
    print(k, func)
#     print(func((1,2,3),))

(0, 9) (<function <lambda> at 0x7fbb69115f70>, -1, 1, -1, 1310, 28, 38)
(0, 17) (<function <lambda> at 0x7fbb6914f5e0>, 1, 1, -1, -1213, 83, 130)
(0, 21) (<function <lambda> at 0x7fbb69115c10>, -1, -1, 1, 1190, -85, -9)
(1, 4) (<function <lambda> at 0x7fbb69115700>, 1, 1, -1, 109, -76, 1111)
(1, 7) (<function <lambda> at 0x7fbb6914fc10>, 1, -1, -1, -1129, 87, 156)
(1, 16) (<function <lambda> at 0x7fbb69115790>, -1, -1, 1, 1231, 114, -60)
(1, 21) (<function <lambda> at 0x7fbb69115ca0>, -1, -1, -1, -1204, -89, 101)
(2, 12) (<function <lambda> at 0x7fbb6914fee0>, 1, -1, -1, 12, 47, -1121)
(2, 17) (<function <lambda> at 0x7fbb6914fe50>, -1, -1, 1, 1209, 9, 113)
(2, 20) (<function <lambda> at 0x7fbb6914fdc0>, 1, -1, 1, 65, 1200, 58)
(3, 6) (<function <lambda> at 0x7fbb6914fa60>, 1, 1, -1, 52, -1314, -56)
(3, 13) (<function <lambda> at 0x7fbb6914f550>, 1, -1, -1, -1177, 42, 85)
(3, 23) (<function <lambda> at 0x7fbb6914fd30>, 1, -1, 1, 5, 17, -1247)
(3, 24) (<function <lambda> at 0x7fbb6914f7

In [7]:
stack = [0]
visited = set(stack)
prev = {}

while stack:
    cur = stack.pop()
    for nex in range(len(srr_inp)):
        if nex in visited:
            continue
        if (cur,nex) not in mapping:
            continue
        stack.append(nex)
        visited.add(nex)
        prev[nex] = cur

In [8]:
visited

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31}

In [9]:
prev

{9: 0,
 17: 0,
 21: 0,
 1: 21,
 11: 21,
 7: 11,
 5: 7,
 15: 5,
 22: 5,
 26: 5,
 23: 22,
 3: 23,
 4: 23,
 14: 23,
 6: 4,
 8: 6,
 27: 8,
 28: 8,
 20: 27,
 2: 20,
 12: 2,
 13: 3,
 24: 3,
 18: 24,
 19: 13,
 25: 13,
 16: 1,
 31: 9,
 10: 31,
 29: 31,
 30: 29}

In [20]:
beacons = set(map(tuple, srr_inp[0]))
scanners = set()

for i in range(len(srr_inp)):
    cur = i
    arr = srr_inp[i]
    brr = [[0,0,0]]
    transforms = []
    
    brr = [[0,0,0]]

    while cur != 0:
        transform, px, py, pz, p9, q9, r9 = mapping[cur, prev[cur]]

        tf = lambda x: (transform((x[0]*px,x[1]*py,x[2]*pz),)[0] - p9, 
                        transform((x[0]*px,x[1]*py,x[2]*pz),)[1] - q9, 
                        transform((x[0]*px,x[1]*py,x[2]*pz),)[2] - r9)
        arr = [tf((x,y,z),) for x,y,z in arr]        
        
        brr = [tf((x,y,z),) for x,y,z in brr]
        cur = prev[cur]
    assert cur == 0

    beacons.update(set(map(tuple, arr)))
    scanners.update(set(map(tuple, brr)))

In [11]:
beacons

{(-4273, -575, -811),
 (-4232, -822, -1774),
 (-4228, -497, -883),
 (-4202, -590, -720),
 (-4168, 637, -683),
 (-4160, 510, -747),
 (-4148, 645, -690),
 (-4123, -671, -1807),
 (-4074, 826, -2029),
 (-4051, -787, -1703),
 (-4035, 835, -1984),
 (-4004, 801, -1857),
 (-3615, 103, -1303),
 (-3587, -44, -1171),
 (-3026, 776, -1769),
 (-2931, -513, -1914),
 (-2912, 835, -1807),
 (-2906, -346, -1875),
 (-2899, 701, -1770),
 (-2880, -380, -390),
 (-2872, -505, -1904),
 (-2860, -522, -443),
 (-2847, -306, -458),
 (-2774, 706, -557),
 (-2674, 607, -605),
 (-2667, 868, -577),
 (-2401, 40, -1315),
 (-2304, 121, -1133),
 (-2003, -1687, -551),
 (-1997, -458, -397),
 (-1966, 870, -1758),
 (-1944, -467, -535),
 (-1942, 2049, -1791),
 (-1939, -499, -325),
 (-1936, 1936, -1674),
 (-1936, 2016, -783),
 (-1922, 697, -1802),
 (-1916, 823, -732),
 (-1898, 2131, -822),
 (-1884, -1768, -636),
 (-1882, 790, -812),
 (-1871, -2663, -1726),
 (-1868, -2729, 589),
 (-1866, 2017, -1783),
 (-1856, -2767, 777),
 (-182

In [12]:
len(beacons)

392

In [ ]:
maxdist = 0
for a,b,c in scanners:
    for x,y,z in scanners:
        maxdist = max(maxdist, abs(a-x) + abs(b-y) + abs(c-z))
maxdist